In [1]:
from api.Kiwoom import *
import sys
from PyQt5.QtWidgets import QApplication

app = QApplication(sys.argv)
kiwoom = Kiwoom()

c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:174: RuntimeWarning: divide by zero encountered in exp2
  eps=exp2(ld(-112)),
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:53: RuntimeWarning: divide by zero encountered in log10
  self.precision = int(-log10(self.eps))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:54: RuntimeWarning: divide by zero encountered in power
  self.resolution = float_to_float(float_conv(10) ** (-self.precision))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:186: RuntimeWarning: divide by zero encountered in exp2
  epsneg_f80 = exp2(ld(-64))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:187: RuntimeWarning: divide by zero encountered in exp2
  tiny_f80 = exp2(ld(-16382))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:201: RuntimeWarn

connected
8046689711 8046689711;


In [17]:
# Buy Order Code
order_result = kiwoom.send_order('send_buy_order', '1001', 1, '252670', 3000, 2540, '00')
print(order_result)

In [2]:
# Get balance
acc_balance = kiwoom.get_balance()
print(acc_balance)

[Kiwoom] _on_receive_msg is called 0002 / opw00018_req / opw00018 / [100000] 모의투자 조회완료
[Kiwoom] _on_receive_tr_data is called 0002 / opw00018_req / opw00018
{'252670': {'종목명': 'KODEX 200선물인버스2X', '보유수량': 4001, '매입가': 2537, '수익률': -2.54, '현재가': 2490, '매입금액': 10150140, '매매가능수량': 4001}}


In [5]:
fids = get_fid("체결시간")
kiwoom.set_real_reg("1000", '252670', fids, "0")

In [6]:
import pandas as pd
close_prices = [100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150]
df = pd.DataFrame(close_prices)
df['ema20'] = df.ewm(span=20, adjust=False).mean()
df.head()

c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\pandas\core\internals\managers.py:226: RuntimeWarning: divide by zero encountered in equal
  if (new_blknos == -1).any():
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\_methods.py:45: RuntimeWarning: divide by zero encountered in reduce
  return umr_any(a, axis, dtype, out, keepdims)
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\pandas\core\window\rolling.py:277: RuntimeWarning: divide by zero encountered in isinf
  inf = np.isinf(values)
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\numeric.py:2327: RuntimeWarning: divide by zero encountered in equal
  return bool(asarray(a1 == a2).all())
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\_methods.py:48: RuntimeWarning: divide by zero encountered in reduce
  return umr_all(a, axis, dtype, out, keepdims)
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\_methods.py:45

,0,ema20
0,100,100.000000
1,105,100.476190
2,110,101.383220
3,115,102.680056
4,120,104.329575


In [7]:
import sqlite3

def insert_df_to_db(db_name, table_name, df, option="replace"):
   with sqlite3.connect('{}.db'.format(db_name)) as con:
       df.to_sql(table_name, con, if_exists=option)

insert_df_to_db("test", 'universe', df)

c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\pandas\core\internals\managers.py:226: RuntimeWarning: divide by zero encountered in equal
  if (new_blknos == -1).any():


In [1]:
from api.Kiwoom import *
from util.make_up_universe import *
from util.db_helper import *
from util.time_helper import *
from util.notifier import *
import math
import traceback
import pandas as pd

class RSIStrategy(QThread):
    def __init__(self):
        QThread.__init__(self)
        self.strategy_name = "RSIStrategy"
        self.kiwoom = Kiwoom()

        # 유니버스 정보를 담을 딕셔너리
        self.universe = {}

        # 계좌 예수금
        self.deposit = 0

        # 초기화 함수 성공 여부 확인 변수
        self.is_init_success = False

        self.init_strategy()

    def init_strategy(self):
        """전략 초기화 기능을 수행하는 함수"""
        try:
            # 유니버스 조회, 없으면 생성
            self.check_and_get_universe()

            # # 가격 정보를 조회, 필요하면 생성
            # self.check_and_get_price_data()

            # # Kiwoom > 주문정보 확인
            # self.kiwoom.get_order()

            # # Kiwoom > 잔고 확인
            # self.kiwoom.get_balance()

            # # Kiwoom > 예수금 확인
            # self.deposit = self.kiwoom.get_deposit()

            # # 유니버스 실시간 체결정보 등록
            # self.set_universe_real_time()

            # self.is_init_success = True

        except Exception as e:
            print(traceback.format_exc())
            # LINE 메시지를 보내는 부분
            send_message(traceback.format_exc(), RSI_STRATEGY_MESSAGE_TOKEN)

    def check_and_get_universe(self):
        """유니버스가 존재하는지 확인하고 없으면 생성하는 함수"""
        if not check_table_exist(self.strategy_name, 'universe'):
            universe_list = get_universe()
            print(universe_list)
            universe = {}
            # 오늘 날짜를 20210101 형태로 지정
            now = datetime.now().strftime("%Y%m%d")

            # KOSPI(0)에 상장된 모든 종목 코드를 가져와 kospi_code_list에 저장
            kospi_code_list = self.kiwoom.get_code_list_by_market("0")

            # KOSDAQ(10)에 상장된 모든 종목 코드를 가져와 kosdaq_code_list에 저장
            kosdaq_code_list = self.kiwoom.get_code_list_by_market("10")

            for code in kospi_code_list+kosdaq_code_list:
                # 모든 종목 코드를 바탕으로 반복문 수행
                code_name = self.kiwoom.get_master_code_name(code)

                # 얻어온 종목명이 유니버스에 포함되어 있다면 딕셔너리에 추가
                if code_name in universe_list:
                    universe[code] = code_name

            #코드, 종목명, 생성일자자를 열로 가지는 DaaFrame 생성
            universe_df = pd.DataFrame(universe.items(), columns=['code', 'code_name'])

        return universe_df
            # universe라는 테이블명으로 Dataframe을 DB에 저장함
        #     insert_df_to_db(self.strategy_name, 'universe', universe_df)

        # sql = "select * from universe"
        # cur = execute_sql(self.strategy_name, sql)
        # universe_list = cur.fetchall()
        # for item in universe_list:
        #     idx, code, code_name, created_at = item
        #     self.universe[code] = {
        #         'code_name': code_name
        #     }
        # print(self.universe)

c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:174: RuntimeWarning: divide by zero encountered in exp2
  eps=exp2(ld(-112)),
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:53: RuntimeWarning: divide by zero encountered in log10
  self.precision = int(-log10(self.eps))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:54: RuntimeWarning: divide by zero encountered in power
  self.resolution = float_to_float(float_conv(10) ** (-self.precision))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:186: RuntimeWarning: divide by zero encountered in exp2
  epsneg_f80 = exp2(ld(-64))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:187: RuntimeWarning: divide by zero encountered in exp2
  tiny_f80 = exp2(ld(-16382))
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\getlimits.py:201: RuntimeWarn

In [2]:
#from strategy.RSIStrategy import *
import sys

app = QApplication(sys.argv)

rsi_strategy = RSIStrategy()
a = rsi_strategy.check_and_get_universe()
a.head()

import sqlite3

def insert_df_to_db(db_name, table_name, df, option="replace"):
   with sqlite3.connect('{}.db'.format(db_name)) as con:
       df.to_sql(table_name, con, if_exists=option)

insert_df_to_db("RSIStrategy", 'universe', a)
app.exec_()

connected
8046689711 8046689711;


c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\pandas\core\internals\managers.py:226: RuntimeWarning: divide by zero encountered in equal
  if (new_blknos == -1).any():
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\_methods.py:45: RuntimeWarning: divide by zero encountered in reduce
  return umr_any(a, axis, dtype, out, keepdims)
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\_methods.py:48: RuntimeWarning: divide by zero encountered in reduce
  return umr_all(a, axis, dtype, out, keepdims)
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\core\fromnumeric.py:90: RuntimeWarning: divide by zero encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: divide by zero encountered in ? (vectorized)
  outputs = ufunc(*inputs)
c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-pa

['LG에너지솔루션', '삼성바이오로직스', '두산에너빌리티', '하이브', '크래프톤', 'LG이노텍', '코스모신소재', '한전기술', 'DB하이텍', '롯데쇼핑', 'LIG넥스원', '현대위아', '솔루엠', '명신산업', 'SK가스', '아이에스동서', '이수페타시스', 'SK리츠', '효성중공업', '해성디에스', '우리종금', 'LF', '빙그레', '백광산업', '케이씨텍', '코리아써키트', 'E1', '에어부산', '한미글로벌', '흥아해운', '잇츠한불', '에스엠벡셀', '동양', '한전산업', '테이팩스', '무림P&P', '화승인더', 'NI스틸', '대영포장', '아시아나IDT', '대구백화점', '한국공항', '토니모리', '다이나믹디자인', '한국화장품제조', '무림페이퍼', '신영와코루', '동원금속', '메타랩스', '상신브레이크', '태평양물산', '천일고속', '한익스프레스', '우진플라임', '갤럭시아에스엠', '모나미']
['LG에너지솔루션', '삼성바이오로직스', '두산에너빌리티', '하이브', '크래프톤', 'LG이노텍', '코스모신소재', '한전기술', 'DB하이텍', '롯데쇼핑', 'LIG넥스원', '현대위아', '솔루엠', '명신산업', 'SK가스', '아이에스동서', '이수페타시스', 'SK리츠', '효성중공업', '해성디에스', '우리종금', 'LF', '빙그레', '백광산업', '케이씨텍', '코리아써키트', 'E1', '에어부산', '한미글로벌', '흥아해운', '잇츠한불', '에스엠벡셀', '동양', '한전산업', '테이팩스', '무림P&P', '화승인더', 'NI스틸', '대영포장', '아시아나IDT', '대구백화점', '한국공항', '토니모리', '다이나믹디자인', '한국화장품제조', '무림페이퍼', '신영와코루', '동원금속', '메타랩스', '상신브레이크', 'DSR', '태평양물산', '천일고속', '한익스프레스', '우진플라임', '갤럭시아에스엠', '모나미']


c:\Users\ihobbang\anaconda3\envs\trade_env\lib\site-packages\pandas\io\sql.py:1526: RuntimeWarning: divide by zero encountered in invert
  col_type = lib.infer_dtype(col, skipna=True)


In [3]:
a.head()

NameError: name 'a' is not defined